In [1]:
!ls drive/MyDrive/Classes/ece542/competition-project/

 2_conv-plus-normalization.ipynb
 attempt-1.ipynb
 best-conv-plus-normalization.ipynb
'Copy of attempt-1.ipynb'
'Copy of conv-plus-normalization.ipynb'
'Copy of current-conv-plus-normalization.ipynb'
'Copy of Hyoer attempt-1.ipynb'
 current-conv-plus-normalization.ipynb
 data.zip
 experiments
'Hyoer attempt-1.ipynb'
 ml_utils
 test_data.zip


In [2]:
!cp drive/MyDrive/Classes/ece542/competition-project/data.zip .

In [3]:
!unzip data.zip

Archive:  data.zip
   creating: data/
   creating: data/preprocessed_data/
   creating: data/splits/
   creating: data/splits/test/
  inflating: data/splits/test/subject_007_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_03__x.csv  
  inflating: data/splits/test/subject_005_session_03__x.csv  
  inflating: data/splits/test/subject_007_session_01__x.csv  
  inflating: data/splits/test/subject_006_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_03__y.csv  
  inflating: data/splits/test/subject_008_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_01__x.csv  
  inflating: data/splits/test/subject_006_session_02__y.csv  
  inflating: data/splits/test/subject_006_session_02__x.csv  
  inflating: data/splits/test/subject_007_session_02__y.csv  
  inflating: data/splits/test/subject_007_session_04__x.csv  
  inflating: data/splits/test/subject_007_session_01__y.csv  
  inflating: data/splits/test/subject_006_session_03__x.csv  


In [4]:
!rm data.zip

In [5]:
# Access token redacted. Replace <ACCESS_TOKEN> with your access token.
!git clone https://savadikarc:<ACCESS_TOKEN>@github.com/savadikarc/ece542-competition-project.git

Cloning into 'ece542-competition-project'...
remote: Enumerating objects: 186, done.
remote: Counting objects: 100% (186/186), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 186 (delta 93), reused 123 (delta 42), pack-reused 0
Receiving objects: 100% (186/186), 266.05 KiB | 14.00 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [6]:
!cd ece542-competition-project && git checkout phase-1 && git pull origin phase-1

Branch 'dataset-perf' set up to track remote branch 'dataset-perf' from 'origin'.
Switched to a new branch 'dataset-perf'
From https://github.com/savadikarc/ece542-competition-project
 * branch            dataset-perf -> FETCH_HEAD
Already up to date.


In [1]:
base_path = "drive/MyDrive/Classes/ece542/competition-project"

In [2]:
import os
import sys
sys.path.append("ece542-competition-project")

In [3]:
import json
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from ml_utils.dataset import SubjectDataset

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [5]:
base_data_path = os.path.join("data", "splits")
train_data_path = os.path.join(base_data_path, "train")
val_data_path = os.path.join(base_data_path, "val")

splits_file = os.path.join(base_path, "experiments", "metadata", "split_ids.json")
with open(splits_file, "r") as f:
    split_ids = json.load(f)

In [6]:
stats_path = os.path.join(base_path, "experiments", "metadata", "statistics.json")
with open(stats_path, "r") as f:
    stats = json.load(f)

In [7]:
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [8]:
min.shape

torch.Size([1, 6, 1])

In [9]:
# Training hyperparameters
batch_size = 128
num_epochs = 20

In [10]:
train_dataset = SubjectDataset(
    train_data_path, 
    split_ids["train"]
)
ys = train_dataset.y.tolist()
counts = Counter(ys)
weights = np.array([1./counts[_y] for _y in ys])
sample_weights = torch.from_numpy(weights).float()
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_iterations = (len(train_dataset) // batch_size) + ((len(train_dataset) % batch_size) != 0)

val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 005_02
Converting uid 001_06
Converting uid 003_02
Converting uid 001_05
Converting uid 002_02
Converting uid 003_01
Converting uid 003_03
Converting uid 005_01
Converting uid 001_07
Converting uid 002_05
Converting uid 004_02
Converting uid 002_03
Converting uid 001_02
Converting uid 002_04
Converting uid 001_03
Converting uid 004_01
Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [11]:
print(train_iterations)
print(val_iterations)

1373
422


In [18]:
# class OneDConvNet(nn.Module):
#   def __init__(self, n_features, n_classes):
#     super(OneDConvNet, self).__init__()
#     self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=16, kernel_size=3, stride=1, padding=1)
#     self.dropout1 = nn.Dropout2d(0.1)
#     self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
#     self.conv2 = nn.Conv1d(in_channels= 16, out_channels=32, kernel_size=3, stride=1, padding=1)
#     self.dropout2 = nn.Dropout2d(0.1)
#     self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
#     self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
#     self.dropout3 = nn.Dropout2d(0.1)
#     self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
#     self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
#     self.dropout4 = nn.Dropout2d(0.1)
#     self.fc1 = nn.Linear(128, 64)
#     self.dropout5 = nn.Dropout(0.2)
#     self.fc2 = nn.Linear(64, n_classes)
  
#   def forward(self, x):
#     x = self.pool1(self.dropout1(F.relu(self.conv1(x))))
#     x = self.pool2(self.dropout2(F.relu(self.conv2(x))))
#     x = self.pool3(self.dropout3(F.relu(self.conv3(x))))
#     x = self.dropout4(F.relu(self.conv4(x)))
#     N, C, T = x.size()
#     x = x.mean(dim=-1) # Flatten
#     x = self.dropout5(F.relu(self.fc1(x)))
#     x = self.fc2(x)
#     return x

# model = OneDConvNet(6, 4).to(device)
class OneDConvNet(nn.Module):
  def __init__(self, n_features, n_classes):
    super(OneDConvNet, self).__init__()
    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=16, kernel_size=3, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(16*40)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels= 16, out_channels=32, kernel_size=3, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(32*20)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(64*10)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(128*5)
    self.fc1 = nn.Linear(128, 64)
    self.dropout5 = nn.Dropout(0.2)
    self.fc2 = nn.Linear(64, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.mean(dim=-1) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

model = OneDConvNet(6, 4).to(device)

In [19]:
# import numpy as np
# X = np.random.randn(1, 6, 40)
# T = torch.from_numpy(X).float().to(device)
# y = model(T)

In [20]:
# y.shape

In [21]:
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.001)
# optimizer = optim.RMSprop(model.parameters(), lr=1e-3)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

In [22]:
def train_step(X, y, model, optimizer, criterion):

    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [23]:
best_val_loss = float("inf")

for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss * X.size(0)
        running_acc = running_acc + corrects

        if iteration % 100 == 0:
            print(f"Iteration: {iteration}/{train_iterations} | train_loss: {loss} | train_acc: {corrects/X.size(0)}")

    train_loss = running_loss / len(train_dataset)
    train_acc = running_acc / len(train_dataset)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss * X.size(0)
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataset)
    val_acc = running_val_acc / len(val_dataset)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        # Checkpoint model
        path = os.path.join(base_path, "experiments", "models", "checkpoint_norm_dropout_32_1.pth")
        print(f"Saving model to {path}")
        torch.save(model.state_dict(), path)
        best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Iteration: 0/1373 | train_loss: 1.3912560939788818 | train_acc: 0.28125
Iteration: 100/1373 | train_loss: 0.9466475248336792 | train_acc: 0.6015625
Iteration: 200/1373 | train_loss: 0.42961451411247253 | train_acc: 0.7734375
Iteration: 300/1373 | train_loss: 0.3682839274406433 | train_acc: 0.765625
Iteration: 400/1373 | train_loss: 0.4608610272407532 | train_acc: 0.765625
Iteration: 500/1373 | train_loss: 0.3002997040748596 | train_acc: 0.8671875
Iteration: 600/1373 | train_loss: 0.256247878074646 | train_acc: 0.8828125
Iteration: 700/1373 | train_loss: 0.295574426651001 | train_acc: 0.90625
Iteration: 800/1373 | train_loss: 0.2684444189071655 | train_acc: 0.890625
Iteration: 900/1373 | train_loss: 0.25935137271881104 | train_acc: 0.8671875
Iteration: 1000/1373 | train_loss: 0.30413317680358887 | train_acc: 0.890625
Iteration: 1100/1373 | train_loss: 0.26311203837394714 | train_acc: 0.9140625
Iteration: 1200/1373 | train_loss: 0.28143614530563354 | train_acc: 0.875
Iteration: 1300/1373

In [24]:
np.array([train_dataset.y[0]])

array([0])

In [37]:
best_model = OneDConvNet(6, 4)
best_model.load_state_dict(torch.load(os.path.join(base_path, "experiments", "models", "checkpoint_norm_dropout_32_1_long.pth")))
best_model = best_model.to(device)

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Evaluation:

  def __init__(self):
        pass
  
  def aggregate_metrics(self, y, y_hat):
    
    """ Metrics for entire set 
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    accuracy  = accuracy_score(y, y_hat)
    precision = precision_score(y, y_hat, average="macro")
    recall    = recall_score(y, y_hat, average="macro")
    f1        = f1_score(y, y_hat, average="macro")
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
        }

  def classwise_metrics(self, y, y_hat):

    """ Metrics for each class. 
        Average is set to macro for calculating the score of each label, and find their unweighted mean. 
        This does not take label imbalance into account.
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    
    class_precision = precision_score(y, y_hat, average=None)
    class_recall    = recall_score(y, y_hat, average=None)
    class_f1        = f1_score(y, y_hat, average=None)
    
    return {
        "Precision": class_precision,
        "Recall": class_recall,
        "F1": class_f1
    }

In [39]:
evaluator = Evaluation()

In [40]:
val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [41]:
output = []
labels = []
for X, y in val_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [42]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [43]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.7935736415412872, 'Precision': 0.7572802478417985, 'Recall': 0.7754044112161469, 'F1': 0.7569545081415515}
Classwise Metrics
{'Precision': array([0.82194655, 0.86334781, 0.6137412 , 0.73008543]), 'Recall': array([0.88291638, 0.87622923, 0.83349819, 0.50897384]), 'F1': array([0.85134126, 0.86974083, 0.70693512, 0.59980083])}


In [44]:
test_data_path = os.path.join(base_data_path, "test")
test_dataset = SubjectDataset(
    test_data_path, 
    split_ids["test"]
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_iterations = (len(test_dataset) // batch_size) + ((len(test_dataset) % batch_size) != 0)

Converting uid 005_03
Converting uid 006_01
Converting uid 006_02
Converting uid 006_03
Converting uid 007_01
Converting uid 007_02
Converting uid 007_03
Converting uid 007_04
Converting uid 008_01


In [45]:
output = []
labels = []
for X, y in test_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [46]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [47]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.8664706660868579, 'Precision': 0.8135768235839757, 'Recall': 0.8872882322514344, 'F1': 0.8451851824617107}
Classwise Metrics
{'Precision': array([0.95271615, 0.80148106, 0.90740741, 0.59270268]), 'Recall': array([0.8735695 , 0.90569681, 0.97350993, 0.79637669]), 'F1': array([0.91142781, 0.85040798, 0.93929712, 0.67960781])}


In [48]:
path = os.path.join(base_path, "experiments", "models", "checkpoint_norm_dropout_32_1_long.pth")
print(f"Saving model to {path}")
torch.save(model.state_dict(), path)

Saving model to drive/MyDrive/Classes/ece542/competition-project/experiments/models/checkpoint_norm_dropout_32_1_long.pth
